In [150]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image, ImageChops
import numpy as np
import math, operator


In [260]:
from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy
import random
import pickle
import pandas as pd
import os.path

PIC = Image.open('template.jpg')
SIZE = 100
VALUES_PER_GENE = 13

NUMBER_OF_TRIANGLES = 3
POPULATION = 4
NGEN = 1000
CXPB = .5
MUTPB = .1

FREQ = 2

NUM_OF_DRAW_OPTIONS = len(["circle","triangle","line","rectangle"])
MAX_CIR_RADIUS = int(SIZE * .1)
LINE_LENGTH = int(SIZE * .15)
LINE_WIDTH = int(SIZE*.01)

MUTATION_NUM_TO_RANDOM = 5

def computeSimilarity(allInds):
    numpyInds = numpy.array(allInds)
    groupedGenes = np.reshape(numpyInds.ravel(order='F'),(NUMBER_OF_TRIANGLES*VALUES_PER_GENE,POPULATION))
    numpy.ndarray.sort(groupedGenes,axis=1)
    
    diffs = numpy.apply_along_axis(np.diff,1,groupedGenes)
    
    multipliers = [i * (POPULATION-i) for i in range(POPULATION)]
    multipliers = multipliers[1:]
    
    scaledDiffs = numpy.apply_along_axis(lambda x: x*multipliers,1,diffs)
    averageDiffInGene = numpy.apply_along_axis(numpy.mean,1,diffs)
    
    return 1-numpy.mean(averageDiffInGene)


def gen_one_triangle():
    return [random.random() for i in range(VALUES_PER_GENE)]


def drawImage(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)
        
        drawShape = int(tri[0] * NUM_OF_DRAW_OPTIONS)
        x1 = int(tri[1] * SIZE)
        y1 = int(tri[2] * SIZE)
        x2 = int(tri[3] * SIZE)
        y2 = int(tri[4] * SIZE)
        x3 = int(tri[5] * SIZE)
        y3 = int(tri[6] * SIZE)
        r = int(tri[7] * 255)
        g = int(tri[8] * 255)
        b = int(tri[9] * 255)
        alpha = int(tri[10] * 40)
        radius = int(tri[11] * MAX_CIR_RADIUS)
        direction = math.radians(int(tri[12]*360))
        
        fillColor = (r,g,b,alpha)
        
        #triangle
        if drawShape == 0:
            triangle = ((x1,y1),(x2,y2),(x3,y3))
            fillColor = (r,g,b,alpha)
            draw.polygon(triangle, fill=fillColor)

        #circle
        if drawShape == 1:
            lowerX,upperX = x1-radius,x1+radius
            lowerY,upperY = y1-radius,y1+radius
            fillColor = (r,g,b,alpha)
            draw.ellipse((lowerX, lowerY, upperX, upperY), fill = fillColor)

        #line
        if drawShape == 2:
            x2 = x1 + math.cos(direction) * LINE_LENGTH
            y2 = y1 + math.sin(direction) * LINE_LENGTH
            draw.line([x1,y1,x2,y2],width=LINE_WIDTH,fill=fillColor)
        
        #rectangle
        if drawShape == 3:
             draw.rectangle([x1,y1,x2,y2],fill=fillColor)
        
        im.paste(mask, mask=mask)
        del mask, draw
    return im


def evaluate(im1, t2):
    im2 = drawImage(t2)

    h = ImageChops.difference(im1,im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r,g,b)]
    err = math.sqrt(sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)) / 256
    return 1 - err,



def mutate(triangles):
    for i in range(MUTATION_NUM_TO_RANDOM):
        triangleToMutate = triangles[randint(0,len(triangles)-1)]
        valueIndexToMutate = randint(0,len(triangleToMutate)-1)
        triangleToMutate[valueIndexToMutate] = random.random()
        
    return triangles,

creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

logbook=None

def main(checkpoint=None):
    if checkpoint and os.path.isfile(checkpoint):
        # A file name has been given, then load the data from the file
        with open(checkpoint, "rb") as cp_file:
            cp = pickle.load(cp_file)
        pop = cp["population"]
        start_gen = cp["generation"]
        hof = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])
    else:
        # Start a new evolution
        pop = toolbox.population(n=POPULATION)
        start_gen = 0
        hof = tools.HallOfFame(maxsize=1)
        logbook = tools.Logbook()

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_sim = tools.Statistics(lambda ind: ind)
    stats_fit.register("std", numpy.std)
    stats_fit.register("max", numpy.max)
    stats_fit.register("avg", numpy.mean)
    stats_fit.register("min", numpy.min)
    stats_sim.register("similarity",computeSimilarity)

    mstats = tools.MultiStatistics(fitness=stats_fit, similarity=stats_sim)

    log = ""
    try:
        for gen in range(start_gen, NGEN):
            pop = algorithms.varAnd(pop, toolbox, cxpb=CXPB, mutpb=MUTPB)

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in pop if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

            hof.update(pop)
            record = mstats.compile(pop)
            logbook.record(gen=gen, evals=len(invalid_ind), **record)
            print(logbook.stream)

            pop = toolbox.select(pop, k=len(pop))

            if gen % FREQ == 0:
                # Fill the dictionary using the dict(key=value[, ...]) constructor
                cp = dict(population=pop, generation=gen, halloffame=hof,
                          logbook=logbook, rndstate=random.getstate())

                with open("checkpoint.pkl", "wb") as cp_file:
                    pickle.dump(cp, cp_file)
                    
    finally:
        df_log = pd.DataFrame(logbook)
        df_log.to_csv('stats.txt', index=False)
        open('result.txt', 'w').write(repr(hof[0]))
        drawImage(hof[0]).save('result.png')
        
if __name__ == '__main__':
    main("checkpoint.pkl")
    #main()

C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


4    	270	0.488354	0.497133	0.462014	0.0152071  	0.977955  
4    	271	0.497133	0.497133	0.497133	0          	1         
0    	272	0.497133	0.497133	0.497133	0          	1         
2    	273	0.497133	0.497133	0.497133	0          	1         
3    	274	0.485321	0.497133	0.449884	0.0204597  	0.987188  
4    	275	0.497133	0.497133	0.497133	0          	1         
2    	276	0.497133	0.497133	0.497133	0          	1         
3    	277	0.487006	0.497133	0.456623	0.0175416  	0.987189  
3    	278	0.466975	0.497133	0.419334	0.0325936  	0.960051  
4    	279	0.497133	0.497133	0.497133	0          	1         
1    	280	0.488645	0.497133	0.463182	0.0147015  	0.986077  
2    	281	0.497133	0.497133	0.497133	0          	1         
0    	282	0.497133	0.497133	0.497133	0          	1         
2    	283	0.497133	0.497133	0.497133	0          	1         
4    	284	0.497133	0.497133	0.497133	0          	1         
2    	285	0.497133	0.497133	0.497133	0          	1         
2    	286	0.497133	0.497133	0.497133	0  

In [188]:
import cProfile
cProfile.run("main()")

   	      	                 fitness                 	similarity
   	      	-----------------------------------------	----------
gen	nevals	avg     	max   	min   	std      	similarity
0  	75    	0.527275	0.5708	0.4831	0.0217533	0.986839  
1  	42    	0.545761	0.580942	0.514339	0.0166201	0.987233  
         1437689 function calls (1314567 primitive calls) in 2.143 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      117    0.007    0.000    0.011    0.000 <ipython-input-187-0c2d7e23367d>:104(<listcomp>)
    30069    0.014    0.000    0.014    0.000 <ipython-input-187-0c2d7e23367d>:105(<genexpr>)
        7    0.000    0.000    0.000    0.000 <ipython-input-187-0c2d7e23367d>:110(mutate)
        1    0.032    0.032    2.141    2.141 <ipython-input-187-0c2d7e23367d>:134(main)
      150    0.000    0.000    0.000    0.000 <ipython-input-187-0c2d7e23367d>:138(<lambda>)
      150    0.000    0.000    0.000    0.000 <ipython-input-18

In [189]:
if 1==0:
    a = [[0.8405457078939849, 0.04661113265122718, 0.9926784556606733, 0.9952372536321301, 0.41085040774137727, 0.4684817318927976, 0.18613858003546313, 0.06201787166612183, 0.05217107641008878, 0.8655018532162592, 0.9085691742848174, 0.3943717113608416, 0.6195190342824293], [0.9381273921168549, 0.9423176248517753, 0.4483793008448148, 0.003875660613077314, 0.764817374994457, 0.623496988540828, 0.4595281804705159, 0.8241386801934246, 0.12950363363540374, 0.22395530214416526, 0.9739542429583149, 0.9688154530449722, 0.48612036540651493], [0.9377948873250016, 0.49599299632530724, 0.8614348107227082, 0.6799167786670913, 0.019099289916788442, 0.8248134699556962, 0.17968472302748195, 0.013836472263828559, 0.06064230523687608, 0.13641277665907525, 0.9075068361666748, 0.5946992948654773, 0.5651893910151318], [0.9564532349064667, 0.989393666872546, 0.9804647786736421, 0.009050647759386465, 0.5199579233481499, 0.6400403252195878, 0.14365232431758757, 0.036964896496684996, 0.10058874448447375, 0.4745750990501735, 0.9800023068989046, 0.8768697803046693, 0.7494489856483874], [0.04159692120581371, 0.122169081811626, 0.39848911450924795, 0.971890622807972, 0.5810265325657297, 0.6025095691487302, 0.2326910373455936, 0.12122619900677667, 0.023289326667939703, 0.23468474964662323, 0.9719350523969099, 0.8722946813219995, 0.5319860585586598], [0.13754493526828837, 0.7209675993043506, 0.9258045300006618, 0.9625348185427535, 0.6178945327898835, 0.07094339357047719, 0.3715844780303158, 0.18769028221806705, 0.014108137308330937, 0.07058481109086379, 0.8223943028747276, 0.8483344356897833, 0.29851147615781437], [0.44561285619003355, 0.9946460059183345, 0.9445005591709796, 0.14540045195590623, 0.1635029643304905, 0.7228640516406658, 0.6916205191561448, 0.06074525625188476, 0.08969884455881538, 0.3842019638513767, 0.9629611617471303, 0.9266477851354992, 0.5331363760457526], [0.405660866756119, 0.41941886357273006, 0.15719632450098198, 0.08932233898922548, 0.6903943427249344, 0.38408863402199356, 0.5597965925311856, 0.07352436997176659, 0.030087786881130563, 0.09984630409439288, 0.9234752967446666, 0.36550066886279575, 0.15862918954079996], [0.7732938284711827, 0.6605802693269948, 0.05946228813919141, 0.39783975765131974, 0.8171345400331173, 0.42788996094507736, 0.44051173026721857, 0.08285590047595537, 0.014141886959789574, 0.0984898777978388, 0.995649144917498, 0.867871489131834, 0.7685085905291957], [0.7635459894316331, 0.9783245235513248, 0.46569686962888024, 0.037755752250183794, 0.736251019566257, 0.5147497217317986, 0.8912048695752602, 0.9231508115226345, 0.025698038408579205, 0.2697000880342405, 0.9601332701744526, 0.44490820107531015, 0.5321933546289714], [0.9500493507940778, 0.5949554495929407, 0.11289996936627822, 0.65070894326035, 0.9958562867471081, 0.9944914640265025, 0.2842823479998645, 0.05054896899337891, 0.005411589111068427, 0.07041719388172862, 0.6508639807585206, 0.09337884478901415, 0.1968454673968989], [0.7796276106853953, 0.6622247850375881, 0.048217834020812256, 0.4019022762769595, 0.5796431965119314, 0.5780766849262418, 0.11496213859818072, 0.06540939896893727, 0.03186776200097219, 0.36608361436858805, 0.8305918048327051, 0.7290075758250082, 0.5070243299137874], [0.314738774392058, 0.9438379579985223, 0.28764370214695445, 0.7617791487458436, 0.7735922392639552, 0.2053700995039497, 0.9667951591032851, 0.2986066726395378, 0.23274263360737724, 0.04454912032950453, 0.489943062955707, 0.9431063140066133, 0.33976807117053476], [0.06348241255567177, 0.5079121681658597, 0.03316300741076639, 0.7941935394251598, 0.8465270210565145, 0.6177219858374857, 0.006300329978694119, 0.01671359528554328, 0.06965597605564611, 0.02268664800577125, 0.9689608382734947, 0.3321835122776722, 0.4155724970361204], [0.05547607837283586, 0.1328972058098412, 0.6665512553151104, 0.867677011182281, 0.37720629767379843, 0.9179510377004695, 0.9928671168393252, 0.38239351364551655, 0.07993185301533523, 0.2545407711454898, 0.685894506383376, 0.002303706995446242, 0.630731708223668], [0.7519100757425774, 0.10415396094463714, 0.6797833145602343, 0.730837607394899, 0.3555687221043482, 0.030420154275115352, 0.23019143102262374, 0.0001658118522246399, 0.021993499030862118, 0.6299690017512909, 0.8777380794969588, 0.91944173099012, 0.3110085847171977], [0.41406069310312565, 0.9748725993961982, 0.9711227780452851, 0.08221354387123192, 0.9606359992919776, 0.6435679630845432, 0.5236791524436377, 0.05848950288903876, 0.14514471990597744, 0.05749466645702728, 0.8916136755499934, 0.908509478772216, 0.5106402787805242], [0.21925793847422026, 0.8093980309607071, 0.39940729376360684, 0.465042183956034, 0.9822793988207038, 0.9761537135373879, 0.7314005138559573, 0.10683335320232479, 0.015244369282156, 0.07375865426077022, 0.8948267377272204, 0.06380643960956733, 0.22590454698134343], [0.8662290761105943, 0.9804016996365116, 0.01940900137695889, 0.3735285533940562, 0.8195828506870237, 0.9046121702166661, 0.49216725547068274, 0.10916090724093686, 0.10760420113501024, 0.24974131638686903, 0.4281460138041234, 0.7350475271028062, 0.45839145185276775], [0.6110269648925745, 0.08781102400658736, 0.35070944744324994, 0.7517418402435478, 0.9449969702907005, 0.6254755429456864, 0.38696083602691933, 0.06024201595002088, 0.19275611276698135, 0.19759566219252944, 0.8928291614676966, 0.09680612433695923, 0.11750124319732613], [0.2743664978450143, 0.9540097493440958, 0.08888030656458723, 0.6319888247659684, 0.48286471930511154, 0.06811753254365716, 0.1867972248327251, 0.6837446251033541, 0.4525283768983296, 0.6204943160344613, 0.9905612322729859, 0.8569621907440498, 0.08425723945539243], [0.7944016107499706, 0.9930839789227128, 0.5137880037071562, 0.3293782912478471, 0.8328734155194907, 0.05822025852082724, 0.5853786068348252, 0.02959130579650593, 0.035605020591092607, 0.040034705844139706, 0.877973412957863, 0.48212980764998803, 0.15990681023081876], [0.14758549295745826, 0.5233861753981768, 0.2863793571283362, 0.7085177317644126, 0.7475950457432488, 0.3301854555999345, 0.11909553338583134, 0.9939920205127457, 0.9445843930011211, 0.8629742899219187, 0.9790402303125135, 0.5363351635772614, 0.8709630316514866], [0.3608155044266744, 0.7889100102490901, 0.7578222786246547, 0.31572345596913276, 0.13757855532110752, 0.801806548622724, 0.05865999806739941, 0.049598563860212774, 0.005793105422866152, 0.011939809203274643, 0.9039718200518349, 0.9143092235561877, 0.31830850835017044], [0.06574164182056041, 0.07855973200223265, 0.8183553039095732, 0.7345529866517184, 0.14274058151927238, 0.4398246253973923, 0.027449344801435682, 0.008862181131492619, 0.03515034206626666, 0.174477500278566, 0.9932336414546734, 0.5272883280489532, 0.1782611048757815], [0.9945625970928195, 0.9947044485891153, 0.46630016012276565, 0.022764125649733402, 0.7345564833098139, 0.29638451262579524, 0.5666629013276118, 0.9199619273315588, 0.024116154480995466, 0.14794962596081296, 0.9829213571292601, 0.2532592586168857, 0.20077758285890412], [0.11640614255104431, 0.5614521152126374, 0.013363600944356957, 0.01032127779968628, 0.6374415374079115, 0.9029386968878566, 0.8364131712298936, 0.062091020297305066, 0.05486732751090584, 0.05043478441910343, 0.9805153547383844, 0.4977200431832779, 0.2606656487067661], [0.2559413423707342, 0.5376906133236437, 0.049310254775780105, 0.3143106111760602, 0.6321474503075004, 0.21285500237103128, 0.5145196244770206, 0.020962995573712173, 0.008881733105197132, 0.018638292194929074, 0.9638173251250396, 0.9138440100898314, 0.20543187690018017], [0.21161618100117996, 0.015038408486083954, 0.8283771275934168, 0.508859837359411, 0.19840223553538427, 0.08628126915265855, 0.4211122940017248, 0.02226491552466492, 0.07968717601174624, 0.22870309431837332, 0.9609334087178845, 0.4823427216084155, 0.5257037042687452], [0.8035574107160696, 0.011219815373140096, 0.6042185332794123, 0.9954038758541424, 0.9909619103270119, 0.8851738950341097, 0.29338282100543134, 0.06054861769286868, 0.0906506700042099, 0.6487195747926662, 0.8573624512966657, 0.9369416620270191, 0.09598282323909424], [0.9899446582471624, 0.6273802317467536, 0.9782099117906705, 0.9911419655395204, 0.5506763801095083, 0.4121649399623597, 0.183080931358783, 0.03591423446608777, 0.03316008373825696, 0.07014694491536166, 0.8836205246478175, 0.2235325491592789, 0.9989549049384241], [0.002479968582391412, 0.5882335189004874, 0.30676451441029784, 0.33480377021004815, 0.17429418545424313, 0.6600344761968768, 0.12583975795688362, 0.9878920768406022, 0.11468320475980198, 0.09724908221554318, 0.8863798927959864, 0.4912040154744428, 0.40074405242127975], [0.03967868565708943, 0.33284817231032837, 0.7735548439084484, 0.11945170274291839, 0.3973642347286912, 0.9477983432593202, 0.9209224345040787, 0.4573988995606548, 0.0018436214794942396, 0.030658577607781234, 0.9470352415347643, 0.027878142916459292, 0.6703135235638843], [0.8633054830866418, 0.9735086878516344, 0.990391539889607, 0.3463872794280515, 0.6828518848204387, 0.44640358813540304, 0.43035050649677353, 0.06521206532909885, 0.00878124671604863, 0.5076518628114802, 0.9996818317788779, 0.6388880410454135, 0.35141258681423293], [0.8673878096557116, 0.9881808834102496, 0.5302376256702934, 0.016369204981580343, 0.6878179992359048, 0.002071202864235566, 0.9711314177844046, 0.9137212398280055, 0.0582104581953764, 0.20438204637806778, 0.8067854286070886, 0.3168752934805297, 0.2681814815753769], [0.8427203926579683, 0.3498692529903621, 0.6344343307566995, 0.04545558419065998, 0.7709902885620666, 0.3404257620804674, 0.362160646476936, 0.033841767155594615, 0.9531403403627291, 0.9988302968367244, 0.8579449875151268, 0.9342590060142278, 0.6444834166360391], [0.8088481637432282, 0.2919420892226693, 0.8407276077042081, 0.9932933036597246, 0.5589066560013702, 0.5198093212711311, 0.6942571306919028, 0.0644501622753505, 0.05107406661561775, 0.03220415612038119, 0.7527411753997525, 0.7215079513066686, 0.5813445338398998], [0.8635832894475334, 0.41740771149514466, 0.1354144881296765, 0.6222468488194665, 0.02433695188203444, 0.9703082225575449, 0.6435788502344385, 0.060091199929979466, 0.014707463536703602, 0.032866841365173816, 0.9506112648791829, 0.5071176932498469, 0.03514845228160113], [0.16263431868432088, 0.6613837681760535, 0.4448518265381466, 0.49361265885699757, 0.14866021641597504, 0.44069955252530557, 0.28891380092044927, 0.802202629044546, 0.7842370748098928, 0.9795809628252145, 0.9713934726996378, 0.2371431248817839, 0.5697601234328473], [0.4098999582728481, 0.7046354341064174, 0.39466873438623873, 0.36391715996128693, 0.5899619187725845, 0.011531328522191187, 0.6248421301244722, 0.849349043688529, 0.06996002236368648, 0.4024338626785453, 0.9052455593990928, 0.63953920966246, 0.13263454747711223], [0.9982930411936849, 0.3659196226613348, 0.6188252257561, 0.008049797318473217, 0.9924637440784696, 0.1305454166118365, 0.18984282083617665, 0.1151604003813268, 0.13037754412957847, 0.8360914753055539, 0.9749245792474789, 0.6446537259654987, 0.48275224401419037], [0.889333693285291, 0.7820497752961472, 0.4382302105262461, 0.09958907845910348, 0.39600383621390745, 0.89139761512907, 0.5528953959725588, 0.01536967107769538, 0.038474711066981304, 0.21994513043452024, 0.9971950841753046, 0.7132030691230397, 0.8457896626015299], [0.1640623106622342, 0.6538018402136937, 0.9401987429519821, 0.7590647462387184, 0.350230114458432, 0.08820199418542973, 0.455335443688221, 0.996591609885043, 0.017314273955647996, 0.2605306425711794, 0.8562041854985869, 0.6429312792690213, 0.8475476209531707], [0.07355833262841527, 0.9680705200892638, 0.39660575024628475, 0.0559333654837354, 0.5005751584237154, 0.4649156864962244, 0.7996987365112868, 0.9505228266245189, 0.22985244659485438, 0.22457341994712732, 0.7529759917597865, 0.48629999742217, 0.7602133261959085], [0.17425646603842004, 0.38434757224634775, 0.8714316936942621, 0.8733818555092289, 0.8380993192415075, 0.9540366948660546, 0.4788172883402705, 0.04152543222577221, 0.010247062177120236, 0.0541767548420905, 0.9378520097997997, 0.8012563260457634, 0.576983002073072], [0.17564864942587932, 0.18658308468747198, 0.388673411871283, 0.9609446674167516, 0.657186117137442, 0.49097146410072123, 0.6106595263041431, 0.11714259662134174, 0.3053649534148847, 0.7955405784803313, 0.9425884681604215, 0.8390720508367937, 0.8057691028851327], [0.8526450680184098, 0.4007534480075621, 0.8002681931427781, 0.6486899687624031, 0.042740845463838606, 0.22376981033382515, 0.09058783288659256, 0.417169639508721, 0.00021844750451915917, 0.011421443093896477, 0.9284783002636916, 0.7324977309919743, 0.918412755997592], [0.7703673890287334, 0.7021308890504283, 0.924073365606726, 0.9854496692644573, 0.8736330579756012, 0.49122495792698684, 0.5562736116898279, 0.8713407582200093, 0.9877769919791847, 0.9888943439469393, 0.9704957374824151, 0.44075104707443524, 0.9502851423725553], [0.8636056290487297, 0.3514525034654794, 0.5401013071047482, 0.8653363168237966, 0.4249182656993866, 0.4262689395296154, 0.6533123275126587, 0.8532309819120577, 0.1586513774444096, 0.49097165155004596, 0.7655958598957151, 0.08886077433230055, 0.7067307761271184], [0.5152777945931964, 0.038040906653015405, 0.9412503387073633, 0.7477359952587974, 0.9046291364006308, 0.2871998580386309, 0.579587937630057, 0.00969332733534256, 0.7010890174510526, 0.9363216038635689, 0.8101441064806338, 0.15894247621050384, 0.7013159430744696], [0.6017023883547995, 0.20021774206710818, 0.3649507724768828, 0.8679298594803064, 0.8580197096282359, 0.3147342243182354, 0.8467998615293235, 0.1089580928357663, 0.15701741161366323, 0.7043293252176395, 0.7164200086062109, 0.8154255935996347, 0.9500552770168486], [0.9937058271080337, 0.009403719246334874, 0.38405522649065327, 0.8737689620982412, 0.9975684586087964, 0.6030282144894095, 0.9688542605065564, 0.03650339506594069, 0.00845490748279687, 0.12241897234611998, 0.7709568498561901, 0.4157305664190082, 0.8714239314383901], [0.7726456926095014, 0.0069492826979913325, 0.7503518593834064, 0.9648191208735722, 0.48695176941622487, 0.7645555749886411, 0.6803620727959243, 0.40982538911173627, 0.005159298488649489, 0.05166968356533963, 0.9777648675263043, 0.6535833985690369, 0.5310050490501943], [0.8293338850689606, 0.03965246819340429, 0.48109606685260686, 0.9908895553310283, 0.6371669768969446, 0.06287921159348198, 0.5289445764261628, 0.9838581218105085, 0.010176296980284105, 0.11084752534648179, 0.9906767003166002, 0.9841122963878279, 0.21516566627933154], [0.08417091613921046, 0.9699551802480162, 0.7642499626988899, 0.23247739837571413, 0.3134354673031341, 0.7684811866254979, 0.3377862616402493, 0.0029341486156876373, 0.07141136673179638, 0.0264576584798204, 0.8872099880848786, 0.5108274412460005, 0.6170248110473452], [0.3317433199866989, 0.35245904998317446, 0.29807730254785403, 0.05216641949103651, 0.38024880168408615, 0.6524050657989631, 0.9366248022076797, 0.0006624855857759737, 0.012230414460808414, 0.3706634810365548, 0.9693846989655679, 0.27381394869552744, 0.8610088697598386], [0.07187527665345639, 0.6540530600385568, 0.8131992575197032, 0.7385323869442395, 0.26947818612820373, 0.9570753651427734, 0.9492949791538634, 0.03212625742280428, 0.09824770350473133, 0.01063393798254697, 0.5992423749209504, 0.9042369710915495, 0.9738220360367403], [0.2962083900780843, 0.7286495395676276, 0.4359754980353453, 0.678048163090296, 0.03336226709681045, 0.7648210203668923, 0.9911105022796082, 0.9828033875168374, 0.08868671834589814, 0.26715605551685684, 0.9154335766609829, 0.8388537754948919, 0.43493011300991513], [0.052011370910183374, 0.002840559413304522, 0.5796633602388599, 0.4064294858926669, 0.20043123067193658, 0.5941335975391941, 0.8794001644233462, 0.02500921107601184, 0.002437507073269507, 0.021086024830513028, 0.9794147686384418, 0.32802709532213703, 0.2719282851741526], [0.23075078550713202, 0.05204434635998112, 0.5357667724865278, 0.9954300582540555, 0.6267575384070616, 0.6199020674356311, 0.27597926342936674, 0.8935469714187426, 0.06435066409061319, 0.009530938823400281, 0.9822624044626704, 0.00901395423402751, 0.6247395617703149], [0.11316161871950403, 0.6659132581820875, 0.1360814203277183, 0.42723966454066564, 0.04264605481987782, 0.33973558047296437, 0.34354748975245974, 0.4257476196628749, 0.07663832505753332, 0.2516022789068716, 0.9937640521858785, 0.7598074040260919, 0.5042990422825264], [0.25006450033414873, 0.8389512686934252, 0.29077276743871205, 0.864498616212357, 0.11618173294497858, 0.24998004552169584, 0.19772217687422033, 0.9070855513530636, 0.6363006960061106, 0.40827086723536554, 0.5087231871609583, 0.15439740430017346, 0.6929952636591508], [0.7546356725729633, 0.9912341100465794, 0.8208163557546393, 0.5092517656591962, 0.0051288255673010985, 0.14645019769057854, 0.005531101307941877, 0.2973826561876938, 0.010522059393162864, 0.06594202679677763, 0.6084559483981604, 0.9380339167035028, 0.5015652674260233], [0.4658923568570613, 0.5836532242360553, 0.6655055974858134, 0.8884713261933315, 0.2220906583927652, 0.8356629330556873, 0.11752669100110735, 0.963162009138179, 0.6194877778429673, 0.7537626379392436, 0.9948924575289997, 0.9881678871173524, 0.6947027407615796], [0.751592997045989, 0.13427314912543997, 0.9218039326198408, 0.32501069865457355, 0.6251663145867403, 0.7702724895834402, 0.5482231463254028, 0.27193324612366265, 0.7795316574264108, 0.9710870836270851, 0.7923858844887426, 0.9355840848361351, 0.5565494534755174], [0.7717119168528863, 0.6060542028498493, 0.5738275918035742, 0.9337140362747022, 0.8312264617414027, 0.019937189535029765, 0.9594658056941446, 0.021236455168115898, 0.05609336225073702, 0.011047832693840798, 0.9913376657957095, 0.01868133282018003, 0.7467488554991438], [0.41415053152012393, 0.6495667099651138, 0.9871149656693503, 0.16304080914852792, 0.7471265599135474, 0.19354517989514586, 0.9169750161383816, 0.29397071502831884, 0.29359188837921424, 0.9157302804857161, 0.7951670869263412, 0.9523196606332142, 0.7408430764783788], [0.40305200190419843, 0.8411942063216576, 0.44776069544662644, 0.617535823195347, 0.12282312461831679, 0.8675780127747241, 0.03143827368774177, 0.7943047954835353, 0.059862076315604695, 0.11907192008956757, 0.9106199279636085, 0.4470655612247041, 0.06887871053409278], [0.0020150019409815867, 0.9773449001239776, 0.4798192963312714, 0.2249965855110272, 0.36855398109313553, 0.6786541258809922, 0.35042694263926477, 0.05788766889629293, 0.005208694890517784, 0.07781171679621313, 0.9701459833504918, 0.37978208415380854, 0.7217858111584846], [0.47563712829785343, 0.44020125621615025, 0.11806067335313797, 0.04093541899368869, 0.8916950904909728, 0.1272368152583907, 0.43372885485681023, 0.04325224016344764, 0.046171942278658396, 0.2829477011085907, 0.9383049848091781, 0.6820981683366206, 0.5098554474693522], [0.44867043171205756, 0.9997805463807746, 0.08625788970252246, 0.706225442678546, 0.6791929943656037, 0.11358946456499275, 0.9101727671887135, 0.047655929341799186, 0.009714318308839509, 0.10193388307368723, 0.9966194473843503, 0.5177596378114927, 0.11986812917710221], [0.45608321643142147, 0.9998625660841249, 0.0859556984358878, 0.8981832111423909, 0.760335974004625, 0.44047252866531283, 0.2809321127567598, 0.006006749022759661, 0.09373037015093943, 0.041846166206649604, 0.9870827105725467, 0.8652595513926079, 0.05507088780925817], [0.4958703080244602, 0.5191323666973465, 0.1746272080681538, 0.34446987185940603, 0.6404372532092412, 0.5531037711200909, 0.8115152694642421, 0.87656059261052, 0.1260086915906249, 0.15878788413636036, 0.9889318839671336, 0.8469834995199895, 0.051348185064129526], [0.032732665826712415, 0.5499814279440719, 0.6558346247964459, 0.7711947148084869, 0.4518486167952638, 0.3942682712606791, 0.523143277505876, 0.9753420783898892, 0.8056953430426141, 0.8357212828879783, 0.7642302600287387, 0.09346137349340555, 0.9107307415065711], [0.08528834657301121, 0.11625040032034883, 0.5356032941950372, 0.03902886187519361, 0.9913626010084661, 0.6302120788657892, 0.9915808783062159, 0.0003956466611595433, 0.5307855305019739, 0.9264891704897589, 0.7557395443942897, 0.9900479657631845, 0.6920199003531091], [0.20633161748430684, 0.9285040077556422, 0.9831461685612375, 0.15155325315457913, 0.957145636641484, 0.996727961788385, 0.7974838583001884, 0.5033313474070281, 0.7720479434292605, 0.9427271289246737, 0.9281590520824055, 0.9997901448493949, 0.277590842943385], [0.7733585781397108, 0.9848098606120314, 0.8490678783192571, 0.027164373607930448, 0.9908014215587577, 0.04749762424280468, 0.9365084664666066, 0.04786630410154846, 0.49891129089642805, 0.9922192413054999, 0.919745306478054, 0.5031257058961409, 0.20148596287856602], [0.38194383479473915, 0.0035270425108321346, 0.6652479767364041, 0.8080980045856846, 0.43431686807133185, 0.036466915326559546, 0.5732427828727729, 0.6212902564839679, 0.029954190510305234, 0.047630648041482315, 0.7705323380351692, 0.7496244554432183, 0.0019230137599066666], [0.2591479417657293, 0.45172636299425917, 0.15219947681743629, 0.002978091749706091, 0.16920318487184283, 0.896434780548331, 0.4090710362603913, 0.1076878593915559, 0.05500129525804898, 0.01471626186345365, 0.36211636933938574, 0.7284911724941839, 0.8259571220095808], [0.03843943988144016, 0.022647568835433396, 0.6803483987208653, 0.9820474457829995, 0.8500755187568524, 0.6098564522528912, 0.04838309984477718, 0.058668841077375244, 0.006217867124726717, 0.0162188861435707, 0.7380147659058914, 0.49607748757146697, 0.1906247668298754], [0.7560562138161496, 0.4004111321487417, 0.5260889385111099, 0.6777652028352037, 0.7215427902615533, 0.6593883798045798, 0.18600591914199094, 0.9440663515217523, 0.26169612464412895, 0.49083193292536464, 0.9706914864542516, 0.05875494031965445, 0.10977510883201713], [0.19235768999302527, 0.08053385638378008, 0.33425368510187226, 0.5460293686264307, 0.8774455411124923, 0.6859571641077875, 0.5658837554225036, 0.9896091473441945, 0.11935349691291208, 0.37256321441873474, 0.9938751735728363, 0.07245161644467502, 0.1655857310275911], [0.7716899292881872, 0.007958216574812105, 0.9819474331323784, 0.6311912035308264, 0.0036750576866458884, 0.04318747560824976, 0.803521954168563, 0.04465772736030371, 0.0004241914318819928, 0.2498438969311122, 0.6163016447631099, 0.8124122679234146, 0.5754810232383059], [0.7893758311784602, 0.7033055673451225, 0.8505932213368398, 0.3257114286990317, 0.4124247292228155, 0.8675352735481441, 0.5197725908843585, 0.8456082847564106, 0.0762650564858276, 0.00568927472391878, 0.7816958661667799, 0.03430649333629476, 0.5965343193579988], [0.9302778543026355, 0.5444005819568887, 0.551318593366488, 0.5897162920794369, 0.6743322569207274, 0.8410935787514391, 0.8673059592940033, 0.9619818734657043, 0.9397839516043275, 0.9118576105675245, 0.958636854433718, 0.8910917451931056, 0.8927172786971657], [0.2617856428962947, 0.5365208370833139, 0.3090834290626342, 0.7844000659823775, 0.8531516587162118, 0.9090710918073366, 0.5750709248110014, 0.8897250047230908, 0.960499782258088, 0.993806873077456, 0.9840789836308126, 0.8803410415203548, 0.01754052822510166], [0.40419533701535826, 0.09724562868969056, 0.98117570086677, 0.12313942182593618, 0.27536605672585357, 0.902319043919744, 0.24310528093655992, 0.21529411373973162, 0.04432612381404788, 0.04867859691688636, 0.9404265375354219, 0.985387036828141, 0.2576554566080339], [0.5775176728837327, 0.004631902166440827, 0.6893955631364235, 0.9938442385621928, 0.7107154724097898, 0.6893054427083268, 0.22313639628251802, 0.09955996260032107, 0.07413021834021027, 0.09222922766253949, 0.9819616297207296, 0.165529814167915, 0.25441019544484367], [0.8542565125861904, 0.06960929404737526, 0.6697887929859131, 0.41711564030388826, 0.5308474682297394, 0.3833934596494295, 0.8145923618741322, 0.048190935121826706, 0.07756316101248417, 0.22330009309930365, 0.965121729653244, 0.32471118170701374, 0.721163110089415], [0.8394082087718201, 0.9309603221728866, 0.5865617580687337, 0.009664285096440017, 0.44372016373080714, 0.9041696245576607, 0.07159084006953009, 0.9809863719331615, 0.0006939523242449619, 0.18808843094359973, 0.9953355812218981, 0.804961804079728, 0.36651110403158504], [0.4105335415051835, 0.17384809100584653, 0.4254480202164741, 0.880592555200031, 0.18165278384408978, 0.0072821891457964805, 0.4247603736150829, 0.08620523495354715, 0.023503201406107288, 0.275298119805903, 0.9142898041637022, 0.31525520358779147, 0.20440959918126256], [0.8186228913827702, 0.6311689623153722, 0.029914089631306262, 0.5592083140761848, 0.46928816054913736, 0.5732445719428921, 0.7250592852149403, 0.040291309652836005, 0.06794625084825501, 0.04258233602861117, 0.9660071590539749, 0.747514770391347, 0.0938459273521044], [0.37236325419138305, 0.10680755388148255, 0.4300955285027419, 0.34089012110201744, 0.1550442331189128, 0.49596883052328056, 0.7483840615652362, 0.22697766592245483, 0.003552489585379237, 0.4396993358188336, 0.6540034691714558, 0.5655813198611077, 0.6655383931310218], [0.9683354002884506, 0.009007551784917522, 0.6043320908198625, 0.9945583339617271, 0.9942599207868044, 0.5650052406556698, 0.8273795118221259, 0.06209005006459589, 0.01568803402987662, 0.02335650030573233, 0.9334339540629816, 0.05823522618307131, 0.4950861259336592], [0.7499991840341564, 0.6178163065497683, 0.26267480406616284, 0.6611602138359698, 0.7643942440363204, 0.6973255834409845, 0.6760821936514656, 0.9911931223747571, 0.4520964315223066, 0.9764575356091204, 0.763957326784738, 0.1036737602812311, 0.7125916767180291], [0.32544456083753526, 0.5032890567122131, 0.3298515677958188, 0.8229314385800659, 0.5801357287667177, 0.774214616183858, 0.46307973538507374, 0.8781493431654888, 0.9245166939690299, 0.9701782095525942, 0.988880268115798, 0.8790246837779316, 0.137906958623356], [0.3209194638795919, 0.1255790918175138, 0.5198278520943888, 0.7211733331427994, 0.10372782304035344, 0.3414139425233931, 0.4161666064909877, 0.9177821924075847, 0.043185980376935995, 0.03501683824458379, 0.9279181568068822, 0.9067930989166348, 0.6220285804824516], [0.9769402877041666, 0.39895610501048884, 0.5845985061666913, 0.004380519967960406, 0.6937368329391768, 0.8220788149110438, 0.010561460074693163, 0.018161997354843584, 0.0019057719379046434, 0.02961437098236075, 0.9403866666625413, 0.5758128380319469, 0.6755326137945432], [0.9012263798777342, 0.6479128669879982, 0.5510429817331062, 0.36638276130089353, 0.6969586713697439, 0.772246264296652, 0.2226293508654048, 0.9450121767997428, 0.2686554947990195, 0.4305220809979774, 0.9865142580770656, 0.9965138063346394, 0.8065388428378234], [0.7589160398577064, 0.6751376920188009, 0.18087333139716621, 0.38586229115295356, 0.8180613118934167, 0.46270916332096734, 0.9472570345432565, 0.8922694417947671, 0.04095718115716218, 0.10233243853717167, 0.9897641712896856, 0.06848918207482069, 0.7560716404487232]]

In [190]:
if 1==0:
    b = numpy.array(a)
    c = b[:,0] * 4
    g = numpy.vectorize(lambda x: int(x))
    c = g(c)
    numpy.ndarray.sort(c,axis=0)
    unique, counts = numpy.unique(c, return_counts=True)
    dict(zip(unique, counts))
